In [72]:
from datetime import datetime, timedelta
import math
from typing import List

def to_seconds(time: timedelta) -> float:
    return time.total_seconds()

def RR(delta_time: float, K_value: float) -> float:
    """Return the retention rate of the user from 0 to 1

    Args:
        delta_time: the time since the last refresh in seconds
        K_value: the K value of the user
    """
    return math.exp(-delta_time / K_value)

def Ktime(Ltime: float, RR_to_refresh: float = 0.8) -> float:
    """Return the K value of the user

    Args:
        Ltime: delta time when the RR will reach RR_to_refresh
        RR_to_refresh: the retention rate to refresh the user
    """
    return -Ltime / math.log(RR_to_refresh)


def Lt(
        refresh_number: int,
        difficulty: float = 1,
        Ltmax: float = 31536000
    ) -> float:
    """Get the Loosing time dependig on the refresh number

    The loosing time is the time it takes for the user to reach RR_to_refresh

    Args:
        refresh_number: the number of refreshes
        S_value: the S value calculated depending on the complexity of the
            element
        Ltmax: the maximum loosing time in seconds, default is 1 year
    """
    difficulty = difficulty if difficulty >= 0 else 0
    return Ltmax / (1 + 132232.47*math.exp(-2.3527 * (refresh_number - difficulty / 3)))



def RetentionRate(
        last_refresh: datetime | timedelta,
        refresh_number: int,
        difficulty: float,
        RR_to_refresh: float = 0.8
    ) -> float:
    """Return the retention rate of the user from 0 to 1"""
    if isinstance(last_refresh, timedelta):
        delta_time = to_seconds(last_refresh)
    elif isinstance(last_refresh, datetime):
        delta_time = to_seconds(datetime.now() - last_refresh)
    else:
        raise TypeError("last_refresh must be a timedelta or a datetime object")
    K_value = Ktime(Lt(refresh_number, difficulty), RR_to_refresh)
    return RR(delta_time, K_value)

In [29]:
LTmax = 60*60*24*365
l_time = [60, 60*60, 60*60*24, 60*60*24*7]

In [79]:
RetentionRate(
    timedelta(days=4),
    refresh_number=3,
    difficulty=0.5,
    RR_to_refresh=0.8) * 100

Lt: 404988.8844979583
345600.0 1814925.3344439748


82.66110105618549

In [1]:
import sys

sys.path.append(r"C:\Users\rango\Desktop\Personale\siapp")
from siapp.db.database import set_work_log

In [2]:
from datetime import datetime, timedelta

t1 = datetime(2024, 11, 8, 8)
t2 = datetime(2024, 11, 8, 10)
t3 = datetime(2024, 11, 8, 10, 12)
t4 = datetime(2024, 11, 8, 11, 31)
t5 = datetime(2024, 11, 8, 12, 45)
t6 = datetime(2024, 11, 8, 16, 1)
t7 = datetime(2024, 11, 8, 16, 30)
t8 = datetime(2024, 11, 8, 17, 15)
l_t = [t1, t2, t3, t4, t5, t6, t7, t8]

for i, t in enumerate(l_t):
    set_work_log(True if i % 2 == 0 else False, t)

In [3]:
t1 = datetime(2024, 11, 9, 8)
t4 = datetime(2024, 11, 9, 11, 31)
t5 = datetime(2024, 11, 9, 12, 45)
t6 = datetime(2024, 11, 9, 16, 1)
t7 = datetime(2024, 11, 9, 16, 30)
t8 = datetime(2024, 11, 9, 17, 15)
l_t = [t1, t4, t5, t6, t7, t8]

for i, t in enumerate(l_t):
    set_work_log(True if i % 2 == 0 else False, t)

In [5]:
from siapp.db.database import analyze_hours

analyze_hours()


[WorkLog(id=1, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 8, 0), check_in=True), WorkLog(id=2, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 10, 0), check_in=False), WorkLog(id=3, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 10, 12), check_in=True), WorkLog(id=4, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 11, 31), check_in=False), WorkLog(id=5, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 12, 45), check_in=True), WorkLog(id=6, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 16, 1), check_in=False), WorkLog(id=7, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 16, 30), check_in=True), WorkLog(id=8, workday_id=1, timestamp=datetime.datetime(2024, 11, 8, 17, 15), check_in=False)]
time_of_work
time_of_work
time_of_work
time_of_work
[WorkLog(id=9, workday_id=2, timestamp=datetime.datetime(2024, 11, 9, 8, 0), check_in=True), WorkLog(id=10, workday_id=2, timestamp=datetime.datetime(2024, 11, 9, 11, 31), check_in=False), WorkLo

In [10]:
from siapp.db.database import get_data_summary

get_data_summary()

[{'date': '2024-11-08 00:00:00',
  'work_time': '7:20:00',
  'lunch_break': '1:14:00',
  'work_break': '0:41:00'},
 {'date': '2024-11-09 00:00:00',
  'work_time': '7:32:00',
  'lunch_break': '1:14:00',
  'work_break': '0:29:00'}]

In [11]:
[str(timedelta(seconds=Lt(i))) for i in range(1, 10)]

NameError: name 'Lt' is not defined

: 